In [23]:
import numpy as np

## Input Parameters

In [24]:
# geometry
L = 1.0
w = 0.05
h = 0.1

# mesh
shape = (100, 10)

# material
E = 1000
nu = 0.2

# load
F = 100

## Analytic Bernoulli Solution

In [25]:
EI = E * w * h**3 / 12
UY_Bernoulli = -F * L**3 / (3 * EI)

## Finite Element Solution

In [26]:
# Hooke model
A = np.array([
    [1, nu, 0], 
    [nu, 1, 0],
    [0, 0, (1-nu)/2]]) * (E /(1-nu**2)) * w

In [27]:
from polymesh.tri.trimesh import TriMesh
trimesh = TriMesh(size=(L, h), shape=shape)
coords = trimesh.coords()
topo = trimesh.topology()

In [28]:
# essential boundary conditions
cond_ebc = np.abs(coords[:, 0]) <= 1e-5
ebcinds = np.where(cond_ebc)[0]
fixity = np.zeros((coords.shape[0], 2), dtype=bool)
fixity[ebcinds, :] = True

In [29]:
# natural boundary conditions
target = np.array([L, h/2, 0])
nbcinds = [trimesh.index_of_closest_point(target)]
loads = np.zeros((coords.shape[0], 2))
loads[nbcinds, 1] = -F

In [30]:
from sigmaepsilon.solid.fem import FemMesh, Structure, CSTM
mesh = FemMesh(coords=coords, topo=topo, celltype=CSTM, 
               fixity=fixity, loads=loads, model=A)
structure = Structure(mesh=mesh)

In [31]:
structure.linsolve()
structure.summary

{'preproc': {'sparsify': False},
 'proc': {'time [ms]': 14.000892639160156,
  'N': 2000,
  'use_umfpack': True,
  'permc_spec': 'COLAMD',
  'solver': 'pardiso'},
 'postproc': {},
 'number of elements': 1782,
 'number of nodes': 1000,
 'dofs per node': 2}

In [32]:
UY_FEM = structure.pointdata.dofsol.to_numpy()[nbcinds[0]][1]

## Comparison

In [33]:
print("Analytic Bernoulli Solution : {}".format(UY_Bernoulli))
print("FEM Solution : {}".format(UY_FEM))
diff = 100 * (UY_FEM - UY_Bernoulli) / UY_Bernoulli
print("Difference : {} %".format(diff))
structure.plot()

Analytic Bernoulli Solution : -7999.999999999998
FEM Solution : -7790.599946023473
Difference : -2.61750067470656 %
